In [ ]:
import cv2
import os

In [ ]:
def preprocess_video(video_path, output_dir, crop_box=None, equalize_hist=False):
    """
    Args:
        video_path (str): Path to input video.
        output_dir (str): Directory to save preprocessed frames.
        crop_box (tuple or None): (x, y, w, h) crop region. If None, no cropping.
        equalize_hist (bool): Apply histogram equalization to grayscale frames.
    """
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Crop if crop_box is given
        if crop_box:
            x, y, w, h = crop_box
            gray = gray[y:y+h, x:x+w]

        # Histogram Equalization
        if equalize_hist:
            gray = cv2.equalizeHist(gray)

        # Save preprocessed frame
        output_path = os.path.join(output_dir, f"frame_{frame_idx:04d}.png")
        cv2.imwrite(output_path, gray)
        frame_idx += 1

    cap.release()
    print(f"Saved {frame_idx} preprocessed frames to: {output_dir}")